In [1]:
from __future__ import division
import sys, os
import numpy as np
import pandas as pd
from collections import OrderedDict
import pyfits

import matplotlib.pyplot as plt
%matplotlib inline

#import matplotlib.mlab as mlab
#from matplotlib.backends.backend_pdf import PdfPages
#from matplotlib.patches import Ellipse

from scipy.stats import norm, lognorm, gamma

In [2]:
#modname      = 'sbpl+blackb'
modname      = 'grbm+blackb'
version      = '-01-'
detector     = 'GBMwLAT'

#version      = 'GBMwLAT_-01-'
#version      = 'GBM_-01-'

burst       = 'bn080916009'

data_dir     = "/Users/derek/Documents/KimiZ/GRBs2/analysis/LAT/%s/integrated/"%burst
mod_dir      = "/Users/derek/Documents/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/%s/%s/"%(burst, detector, modname)
desk_dir     = "/Users/derek/Desktop/kim/"
#mod_dir         = "/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/%s/%s/"%(burst, detector, modname)
#data_dir        = "/Users/KimiZ/GRBs2/analysis/LAT/%s/integrated/"%burst

fitdatafile       = os.path.join(mod_dir, "xspec_fitresults_%s_%s.fit"%(modname, version))
fakeitdatafile    = os.path.join(data_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))

outfilename = "AfterFakeit_ParamDistPlots_%s_%s.pdf"%(modname, version)
outfile1 = os.path.join(mod_dir, outfilename)
outfile2 = os.path.join(desk_dir, outfilename)

paramoutfile1 = os.path.join(mod_dir, "AfterFakeit_ParamErrors_%s_%s.txt"%(modname, version))
paramoutfile2 = os.path.join(mod_dir, "AfterFakeit_ParamErrorsEASYREAD_%s_%s.txt"%(modname, version))

In [3]:
f2 = pyfits.open(fakeitdatafile )

iter = 0
fakeparams = pd.DataFrame()
for name in f2[1].data.columns.names:
    fakeparams["%s"%name] = f2[1].data["%s"%name]
    if 'PARAM' in name:
        iter += 1
    else:
        pass
nPars = iter

FakeParams = fakeparams.astype('float64')

In [4]:
f1 = pyfits.open(fitdatafile)
param_dict = OrderedDict()
for name in f1[2].data.columns.names:
    param_dict["%s"%name] = f1[2].data["%s"%name][0]

In [5]:
def check_aic_bic(Cstat, Npars, Ndata=None):
    import numpy as np
    Aic = Cstat + 2.*Npars
    Bic = Cstat + Npars * np.log(Ndata)
    return Aic, Bic

In [6]:
aic = param_dict.get("AIC")
bic = param_dict.get("BIC")
cstat = param_dict.get("CSTAT")
ndata = param_dict.get("NDATA")
covarmat = param_dict.get("COVARMAT")
# nPars # found earlier

In [7]:
if len(covarmat) != nPars**2:
    print "***"*20
    print "*** WARNING: old covarmat.  Has %i elements instead of %i."%(len(covarmat), nPars**2)
    print "***"*20

************************************************************
*** WARNING: old covarmat.  Has 21 elements instead of 36.
************************************************************


In [8]:
AIC, BIC = check_aic_bic(cstat, nPars, ndata)

if AIC != aic:
    raise Warning, "AIC's don't match"
    #raise Exception, "AIC's don't match"

if BIC != bic:
    raise Warning, "BIC's don't match"
    #raise Exception, "BIC's don't match"

In [48]:
def Compare_Distributions(parameter, norm_dict_, lognorm_dict_, gamma_dict_):
    data = abs(FakeParams["%s"%parameter])
    print parameter
    
    distributions = [norm, lognorm, gamma]
    mles = []
    for distribution in distributions:
        pars = distribution.fit(data)
        errors = distribution.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], *pars)
        mle = distribution.nnlf(pars, data)
        mles.append(mle)
        parlist = str(pars)
        #values = exec("%s.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], %s)"%(distribution.name, pars))#%(distribution, pars))
        print "{}   {} ".format(distribution.name, pars)
        print "{} errors:  {}".format(distribution.name, errors)
        if 'log' in distribution.name:
            lognorm_dict_.update({parameter: [pars, errors]})
        elif 'gamma' in distribution.name:
            gamma_dict_.update({parameter: [pars, errors]})
        else:
            norm_dict_.update({parameter: [pars, errors]})
            

    results = [(distribution.name, mle) for distribution, mle in zip(distributions, mles)]
    best_fit = sorted(zip(distributions, mles), key=lambda d: d[1])[0]
    print "Best fit:   {}".format(best_fit[0].name)
    print "MLE value:  {}".format(best_fit[1])
    print "results:    {}".format(results)
    print "difference between MLEs:   {}".format(np.diff([results[0][1], results[1][1]])[0])
    print "\n"

In [49]:
norm_dict = OrderedDict()
lognorm_dict = OrderedDict()
gamma_dict = OrderedDict()

for pname in FakeParams.columns:
    if 'PARAM' in pname:
        Compare_Distributions(pname, norm_dict, lognorm_dict, gamma_dict)
    elif ('AIC' in pname) or ('BIC' in pname) or ('LOGLIKE' in pname) or ('CSTAT' in pname):
        Compare_Distributions(pname, norm_dict, lognorm_dict, gamma_dict)
    else:
        pass

PARAM1
norm   (1.2189943141050157, 0.033993967678901393) 
norm errors:  [ 1.18499961  1.25298902  1.1510063   1.28698233  1.11701319  1.32097544]
lognorm   (0.021004598427657895, -0.39171960573642539, 1.6100273403074832) 
lognorm errors:  [ 1.18484173  1.25248413  1.15207268  1.28738455  1.11998493  1.32302555]
gamma   (386.65569499706578, 0.54298053189668405, 0.0017480062613529791) 
gamma errors:  [ 1.18449921  1.25321513  1.15187545  1.28933412  1.1203866   1.32664692]
Best fit:   norm
MLE value:  -19626.3365811
results:    [('norm', -19626.336581110903), ('lognorm', -19593.730824575716), ('gamma', -19571.708656226554)]
difference between MLEs:   32.6057565352


PARAM2
norm   (2.2309670278750868, 0.024629233163413585) 
norm errors:  [ 2.20633726  2.2555968   2.1817085   2.28022555  2.1570799   2.30485416]
lognorm   (0.10451635056906655, 1.9959714229835424, 0.23371498983876238) 
lognorm errors:  [ 2.20649208  2.25543621  2.1856004   2.28402179  2.16678209  2.31575647]
gamma   (41.9636

In [ ]:
valsA = norm.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], p1, p2)

In [ ]:
def Plot_ParamDistributions(modelname, parameter, norm_dict_, lognorm_dict_, gamma_dict_, Xlimits=None, bins=100):
    data = abs(FakeParams["%s"%parameter])
    
    # distribution_dict = OrderedDict()
    # distributions = [norm, lognorm]
    # mles = []
    # for distribution in distributions:
    #     pars = distribution.fit(data)
    #     mle = distribution.nnlf(pars, data)
    #     mles.append(mle)
    #     distribution_dict[distribution.name] = pars # save pars into a dictionary
    #     print "\n"
    #     print "Distribution: {}".format(distribution.name)
    #     print "Parameters:   {}".format(pars)

    # results = [(distribution.name, mle) for distribution, mle in zip(distributions, mles)]
    # best_fit = sorted(zip(distributions, mles), key=lambda d: d[1])[0]
    # print ""
    # print "Best fit:   {}".format(best_fit[0].name)
    # print "MLE value:  {}".format(best_fit[1])
    # print "results:    {}".format(results)
    # print "difference between MLEs:   {}".format(np.diff([results[0][1], results[1][1]])[0])
    # print "\n"
    
    plt.figure(figsize = (11,7))
    counts, bins, ignored = plt.hist(data, bins=100, normed=True, align='mid', alpha=0.2)
    
    x = np.linspace(min(bins), max(bins), 1000)
    
    p1, p2 = norm_dict_.get(parameter)
    #p1, p2 = norm.fit(data)
    #p1, p2 = distribution_dict.get('norm')
    plt.plot(x, norm.pdf(x, p1, p2), 'b-', lw=2, label="Normal FIT")

    #p3, p4, p5 = lognorm.fit(data)
    #p3, p4, p5 = distribution_dict.get('lognorm')
    p3, p4, p5 = lognorm_dict_.get(parameter)
    plt.plot(x, lognorm.pdf(x, p3, p4, p5), 'r-', lw=2, label="Lognormal FIT")
    
    p6, p7, p8 = gamma_dict_.get(parameter)
    plt.plot(x, gamma.pdf(x, p6, p7, p8), 'y-', lw=2, label="Gamma FIT")

    # ylow, yhi = ax.get_ylim()
    # ylabelpos = yhi - (yhi*.01)
    # yhigh = yhi + (yhi*.1)
    mu, sigma = p1,p2
    
    valsC = [mu-sigma, mu+sigma, mu-2*sigma, mu+2*sigma, mu-3*sigma, mu+3*sigma]
    plt.vlines(valsC, 0, counts.max(), linestyle='-', color='yellow')
    
    valsA = norm.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], p1, p2)
    valsB = lognorm.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], p3, p4, p5)

    plt.vlines(valsA, 0, counts.max(), linestyle='--', color='blue')
    plt.vlines(valsB, 0, counts.max(), linestyle='--', color='red')
    
    if Xlimits is not None:
        plt.xlim(Xlimits)
    
    plt.ylabel("Frequency")
    plt.xlabel('%s'%parameter)
    plt.legend(loc='best')
    plt.title('%s'%modelname)


In [ ]:
for pname in FakeParams.columns:
    if 'PARAM' in pname:
        Plot_ParamDistributions(modname, pname, norm_dict, lognorm_dict)
        #plt.savefig(pp, format='pdf', dpi=150, bbox_inches = 'tight')
    elif ('AIC' in pname) or ('BIC' in pname) or ('LOGLIKE' in pname) or ('CSTAT' in pname):
        Plot_ParamDistributions(modname, pname, norm_dict, lognorm_dict)
        #plt.savefig(pp, format='pdf', dpi=150, bbox_inches = 'tight')
    else:
        pass